In [78]:
# 安装用到的库
import json
import numpy as np

In [79]:
# 这个类的目的是把json文件读入成字典格式，该类把json文件中的size，A，D和M分别
# 设置为类的四个属性。
class InputData:
    def __init__(self, path):
        self.__path = path
        self.DataLoad()

    def DataLoad(self):
        with open(self.__path, "r") as inputFile:
            inputData = json.load(inputFile)

        self.Size = inputData["Size"]

        # 使用数组是为了更加直观np.array()
        self.Transport_Matrix = np.array(inputData['A']) # 运输数量矩阵，从设备 i 到设备 j
        self.Distance_Matrix = np.array(inputData['D'])  # Station之间的距离矩阵，从k到l
        self.Mind_Entfernung = np.array(inputData['M'])  # 设备Anlage i和j之间，至少应保持的距离

    def __str__(self):
        # str函数用于直接打印inputData对象时，输出这个json文件涉及多少个设备的排列问题
        return f'Quadratic Assignment Problem von {self.Size} Anlagen. '




In [81]:
data = InputData('scr12.json')
print(data.Size)
print(data)
print(data.Distance_Matrix)
print(type(data.Distance_Matrix))
print(data.Distance_Matrix[0][1])

12
Quadratic Assignment Problem von 12 Anlagen. 
[[0 1 2 3 1 2 3 4 2 3 4 5]
 [1 0 1 2 2 1 2 3 3 2 3 4]
 [2 1 0 1 3 2 1 2 4 3 2 3]
 [3 2 1 0 4 3 2 1 5 4 3 2]
 [1 2 3 4 0 1 2 3 1 2 3 4]
 [2 1 2 3 1 0 1 2 2 1 2 3]
 [3 2 1 2 2 1 0 1 3 2 1 2]
 [4 3 2 1 3 2 1 0 4 3 2 1]
 [2 3 4 5 1 2 3 4 0 1 2 3]
 [3 2 3 4 2 1 2 3 1 0 1 2]
 [4 3 2 3 3 2 1 2 2 1 0 1]
 [5 4 3 2 4 3 2 1 3 2 1 0]]
<class 'numpy.ndarray'>
1


In [82]:
data_15 = InputData("scr15.json")
data_20 = InputData("scr20.json")
data_25 = InputData("tai25b.json")

In [26]:
## 验证对称性
def checkSymA(data):
    result = 0
    for i in range(data.Size):
        for j in range(data.Size):
            if data.Transport_Matrix[i][j] != data.Transport_Matrix[j][i]:
                result += 1
                
            else:
                result += 0

    return result

def checkSymD(data):
    result = 0
    for i in range(data.Size):
        for j in range(data.Size):
            if data.Distance_Matrix[i][j] != data.Distance_Matrix[j][i]:
                result += 1
                
            else:
                result += 0

    return result

def checkSymM(data):
    result = 0
    for i in range(data.Size):
        # 因为矩阵的行数等于列数，也就是size
        for j in range(data.Size):
            if data.Mind_Entfernung[i][j] != data.Mind_Entfernung[j][i]:
                result += 1
                
            else:
                result += 0

    return result

In [27]:
# 经过下面三个单元格的输出可知，4个文件中的A矩阵都是对称的，
# 25台设备的情况下，D矩阵和M矩阵不对称，D不对称是由单行线导致，M不对称可能是不同机器的对最近距离的要求不同。
# scr12，15和20的情况下，三个矩阵都是对称的，解决问题时只需要考虑是否要将最近距离M考虑在内，可以用子类和父类来做
# 父类是不考虑机器之间应保持的最近距离，只用A和D矩阵来做，而子类需要重写父类的运算过程，加一个if，判断
# 对应位置的距离D和对应机器的最小距离M哪个更大，更大的那个用来做计算。这个计算过程不一定要一步一步算，可以先来
# 构建一个用于计算的距离矩阵（这个矩阵包含D中和M中更大的那个值），然后用这个矩阵和A矩阵做矩阵相乘, 然后用.sum()
print(checkSymA(data))
print(checkSymA(data_15))
print(checkSymA(data_20))
print(checkSymA(data_25))

0
0
0
0


In [28]:
print(checkSymD(data))
print(checkSymD(data_15))
print(checkSymD(data_20))
print(checkSymD(data_25))

0
0
0
498


In [29]:
print(checkSymM(data))
print(checkSymM(data_15))
print(checkSymM(data_20))
print(checkSymM(data_25))

0
0
0
36


In [84]:
# 矩阵或数组之间相乘，得到的结果是对应位置元素之间相乘的结果
a = np.array([[1,2], [3,2]])
b = np.array([[2,4], [1,3]])
print(a*b)
print((a*b).sum())
print(sum(a*b))
print(len(a))
print(len(a*b))

[[2 8]
 [3 6]]
19
[ 5 14]
2
2


In [31]:
print(list(range(data.Size)))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]


In [32]:
gesamt_12 = data.Distance_Matrix * data.Transport_Matrix
print(gesamt_12.sum())
print(sum(gesamt_12))

50116
[1180 8742 3208 9982 1755 2902 8129 2805 2790 4874  709 3040]


In [33]:
import copy
robot=[1,2,3]
see=[]
see=copy.deepcopy(robot)
print(see)

[1, 2, 3]


In [85]:
# 这个类是将输入的数据进行预处理，以输出gesamtTransportLeistung（分两种情况）作为属性。
# 类似讲义中的ProcessingTime
# 用inputData作为输入
import copy
class OutputData:
    def __init__(self, inputData, withMinimumDistance = False):
        self.InputData = inputData
        self.Size = self.InputData.Size
        self.A = self.InputData.Transport_Matrix
        self.D = self.InputData.Distance_Matrix
        self.M = self.InputData.Mind_Entfernung

        # InitialPermutation 是对Station的排列，可理解为把这个Station送给某个机器  -->重新排列后的D矩阵在Solution类中做！！！！
        self.InitialPermutation = list(range(self.InputData.Size)) # 这里输出的是初始排列，即fcfs。因此这里先不用考虑对Station的排列
        self.WithMinimumDistance = withMinimumDistance
        self.gesamtTransportLeistung = self.CalculateInitialTransport()
        
        

    def CalculateInitialTransport(self):
        if self.WithMinimumDistance == False:
            total = (self.A * self.D).sum()
        else:
            # 当考虑设备间的最短距离时，需要比较D矩阵和M矩阵对应元素的大小，留下大的。
            # 通过观察json中的D和M，M是个稀疏矩阵，因此我只需要创建一个D_copy，把M更大的元素赋给这个副本矩阵，，然后用副本矩阵跟A相乘
            DistanceWithMin = copy.deepcopy(self.D)

            # fcfs的情况下，0号设备放在0号Station，并以此类推，因此不用考虑对矩阵D的重新排列
            # Solution中需要先把D按照Permutation排列好，然后再计算
            for i in self.InitialPermutation:
                for j in self.InitialPermutation:
                    if self.D[i][j] < self.M[i][j]:
                        DistanceWithMin[i][j] = self.M[i][j]
            total = (self.A * DistanceWithMin).sum()
        return total


    def __str__(self):
        return f'OutputData: \nQuadratic Assignment Problem von {self.InputData.Size} Anlagen, mit einer initiallen Transportleistung von {self.gesamtTransportLeistung}.'
        
output_12 = OutputData(data)
print(len(output_12.D))
print(output_12)

output_12_m = OutputData(data, True)
print(output_12_m)

print(output_12.D)


12
OutputData: 
Quadratic Assignment Problem von 12 Anlagen, mit einer initiallen Transportleistung von 50116.
OutputData: 
Quadratic Assignment Problem von 12 Anlagen, mit einer initiallen Transportleistung von 50998.
[[0 1 2 3 1 2 3 4 2 3 4 5]
 [1 0 1 2 2 1 2 3 3 2 3 4]
 [2 1 0 1 3 2 1 2 4 3 2 3]
 [3 2 1 0 4 3 2 1 5 4 3 2]
 [1 2 3 4 0 1 2 3 1 2 3 4]
 [2 1 2 3 1 0 1 2 2 1 2 3]
 [3 2 1 2 2 1 0 1 3 2 1 2]
 [4 3 2 1 3 2 1 0 4 3 2 1]
 [2 3 4 5 1 2 3 4 0 1 2 3]
 [3 2 3 4 2 1 2 3 1 0 1 2]
 [4 3 2 3 3 2 1 2 2 1 0 1]
 [5 4 3 2 4 3 2 1 3 2 1 0]]


In [35]:
D_c = np.array([[0,2,2,3,1,2,3,4,2,3,4,5],
[2,0,1, 2, 2, 1, 2, 3, 3, 2, 3, 4], [2, 1, 0, 1, 3, 2, 1, 2, 4, 3, 2, 3], [3, 2, 1, 0, 4, 3, 2, 1, 5, 4, 3, 2], [1, 2, 3, 4, 0, 1, 2, 3, 1, 2, 3, 4],
        [2, 1, 2, 3, 1, 0, 1, 2, 2, 1, 2, 3],
        [3, 2, 1, 2, 2, 1, 0, 1, 3, 2, 1, 2],
        [4, 3, 2, 1, 3, 2, 1, 0, 4, 3, 2, 1],
        [2, 3, 4, 5, 1, 2, 3, 4, 0, 1, 2, 3],[3, 2, 3, 4, 2, 1, 2, 3, 1, 0, 2, 2],[4, 3, 2, 3, 3, 2, 1, 2, 2, 2, 0, 2],[5, 4, 3, 2, 4, 3, 2, 1, 3, 2, 2, 0]])


print((D_c * data.Transport_Matrix).sum())

50998


In [36]:
# 矩阵和二维数组：按照permutation重新排列行和列

Dist= np.array([[1,2,3], [4,5,6],[7,8,9]])
new_perm = [2,0,1]
print(Dist)
# new_perm = np.array(range(12)[::-1])
# print(new_perm)
dist_zeil = Dist[new_perm, :] # 先把Dist的行按照perm进行排列，
print(dist_zeil)

dist_zeil = dist_zeil[:, new_perm] # 再把列进行从新排列
print(dist_zeil)


[[1 2 3]
 [4 5 6]
 [7 8 9]]
[[7 8 9]
 [1 2 3]
 [4 5 6]]
[[9 7 8]
 [3 1 2]
 [6 4 5]]


In [86]:
res = 0
for i in range(12):
    for j in range(12):
        if output_12.D[i][j] < output_12.M[i][j]:
            res += 1

print(res)


print(list(range(12)))

6
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]


In [89]:
# Permutation 是指station的排序；anlage矩阵不排序，默认从1（0）到n（n-1）排序。设备的排序也就相当于把这些设备分别安排在相应的Station中
class Solution:

    def __init__(self, outputData, permutation):
        self.OutputData = outputData
        self.Permutation = permutation
        self.TransportMatrix = self.OutputData.A
        self.ArrangedDMatrix = self.ArrangeDistanceMatrix() # 按照Permutation调整D矩阵的行列，得到Anlagen之间的距离，然后再与M矩阵直接比较
        self.UpdatedDMatrix = self.UpdateDMatrix() # 

        self.TotalTransport = -1  # 需要用EvaluationLogic类来计算具体的值

    def __str__(self):
        return f'Die Permutation {self.Permutation} führt zu einer Transportleistung von {self.TotalTransport}'

    def setPermutation(self, permutation):
        self.Permutation = permutation

    # 按照permutation重新排列D矩阵的行和列
    def ArrangeDistanceMatrix(self):
        arrangedDMatrix = self.OutputData.D[self.Permutation, :]
        arrangedDMatrix = arrangedDMatrix[:, self.Permutation]

        return arrangedDMatrix

    # 通过比较当前D矩阵和最小距离M矩阵，把得出更新后的D矩阵
    def UpdateDMatrix(self):
        DistanceWithMin = copy.deepcopy(self.ArrangedDMatrix)
        # res = 0
        for i in range(self.OutputData.Size):
            for j in range(self.OutputData.Size):
                if self.ArrangedDMatrix[i][j] < self.OutputData.M[i][j]:
                    DistanceWithMin[i][j] = self.OutputData.M[i][j]
                    # res += 1
        # print(res)
        return DistanceWithMin
        

sol_1 = Solution(output_12, list(range(12))) 
print(sol_1.ArrangedDMatrix)
print(sol_1.UpdatedDMatrix)


# 得出结论，我自己随便找的一个排列，会导致fcfs中的6个不满足最近距离的元素变成4个






[[0 1 2 3 1 2 3 4 2 3 4 5]
 [1 0 1 2 2 1 2 3 3 2 3 4]
 [2 1 0 1 3 2 1 2 4 3 2 3]
 [3 2 1 0 4 3 2 1 5 4 3 2]
 [1 2 3 4 0 1 2 3 1 2 3 4]
 [2 1 2 3 1 0 1 2 2 1 2 3]
 [3 2 1 2 2 1 0 1 3 2 1 2]
 [4 3 2 1 3 2 1 0 4 3 2 1]
 [2 3 4 5 1 2 3 4 0 1 2 3]
 [3 2 3 4 2 1 2 3 1 0 1 2]
 [4 3 2 3 3 2 1 2 2 1 0 1]
 [5 4 3 2 4 3 2 1 3 2 1 0]]
[[0 2 2 3 1 2 3 4 2 3 4 5]
 [2 0 1 2 2 1 2 3 3 2 3 4]
 [2 1 0 1 3 2 1 2 4 3 2 3]
 [3 2 1 0 4 3 2 1 5 4 3 2]
 [1 2 3 4 0 1 2 3 1 2 3 4]
 [2 1 2 3 1 0 1 2 2 1 2 3]
 [3 2 1 2 2 1 0 1 3 2 1 2]
 [4 3 2 1 3 2 1 0 4 3 2 1]
 [2 3 4 5 1 2 3 4 0 1 2 3]
 [3 2 3 4 2 1 2 3 1 0 2 2]
 [4 3 2 3 3 2 1 2 2 2 0 2]
 [5 4 3 2 4 3 2 1 3 2 2 0]]


In [90]:
# 把第11台设备放在了第一个Station     [0,11,1,10,2,9,3,8,4,7,5,6]
sol_2 = Solution(output_12, [0,11,1,10,2,9,3,8,4,7,5,6])
print(sol_2.ArrangedDMatrix)
print(sol_2.UpdatedDMatrix)

[[0 5 1 4 2 3 3 2 1 4 2 3]
 [5 0 4 1 3 2 2 3 4 1 3 2]
 [1 4 0 3 1 2 2 3 2 3 1 2]
 [4 1 3 0 2 1 3 2 3 2 2 1]
 [2 3 1 2 0 3 1 4 3 2 2 1]
 [3 2 2 1 3 0 4 1 2 3 1 2]
 [3 2 2 3 1 4 0 5 4 1 3 2]
 [2 3 3 2 4 1 5 0 1 4 2 3]
 [1 4 2 3 3 2 4 1 0 3 1 2]
 [4 1 3 2 2 3 1 4 3 0 2 1]
 [2 3 1 2 2 1 3 2 1 2 0 1]
 [3 2 2 1 1 2 2 3 2 1 1 0]]
[[0 5 1 4 2 3 3 2 1 4 2 3]
 [5 0 4 1 3 2 2 3 4 1 3 2]
 [1 4 0 3 1 2 2 3 2 3 2 2]
 [4 1 3 0 2 1 3 2 3 2 2 1]
 [2 3 1 2 0 3 1 4 3 2 2 1]
 [3 2 2 1 3 0 4 1 2 3 1 2]
 [3 2 2 3 1 4 0 5 4 1 3 2]
 [2 3 3 2 4 1 5 0 1 4 2 3]
 [1 4 2 3 3 2 4 1 0 3 1 2]
 [4 1 3 2 2 3 1 4 3 0 2 1]
 [2 3 2 2 2 1 3 2 1 2 0 2]
 [3 2 2 1 1 2 2 3 2 1 2 0]]


In [91]:
sol_3 = Solution(output_12, [0,2,4,6,8,10,11,9,7,5,3,1])
print(sol_3.ArrangedDMatrix)
print(sol_3.UpdatedDMatrix)

[[0 2 1 3 2 4 5 3 4 2 3 1]
 [2 0 3 1 4 2 3 3 2 2 1 1]
 [1 3 0 2 1 3 4 2 3 1 4 2]
 [3 1 2 0 3 1 2 2 1 1 2 2]
 [2 4 1 3 0 2 3 1 4 2 5 3]
 [4 2 3 1 2 0 1 1 2 2 3 3]
 [5 3 4 2 3 1 0 2 1 3 2 4]
 [3 3 2 2 1 1 2 0 3 1 4 2]
 [4 2 3 1 4 2 1 3 0 2 1 3]
 [2 2 1 1 2 2 3 1 2 0 3 1]
 [3 1 4 2 5 3 2 4 1 3 0 2]
 [1 1 2 2 3 3 4 2 3 1 2 0]]
[[0 2 1 3 2 4 5 3 4 2 3 1]
 [2 0 3 1 4 2 3 3 2 2 1 1]
 [1 3 0 2 1 3 4 2 3 2 4 2]
 [3 1 2 0 3 1 2 2 1 1 2 2]
 [2 4 1 3 0 2 3 1 4 2 5 3]
 [4 2 3 1 2 0 1 1 2 2 3 3]
 [5 3 4 2 3 1 0 2 1 3 2 4]
 [3 3 2 2 1 1 2 0 3 1 4 2]
 [4 2 3 1 4 2 1 3 0 2 1 3]
 [2 2 2 1 2 2 3 1 2 0 3 1]
 [3 1 4 2 5 3 2 4 1 3 0 2]
 [1 1 2 2 3 3 4 2 3 1 2 0]]


In [92]:
class SolutionPool:
    def __init__(self):
        self.Solutions = []

    def AddSolution(self, newSolution):
        self.Solutions.append(newSolution)

    def GetLowestTransport(self):
        self.Solutions.sort(key = lambda solution: solution.TotalTransport)

        return self.Solutions[0]

In [117]:
class EvaluationLogic:
    # def __init__(self, inputData):
    #     self.InputData = inputData
    #     self.TotalTransport = None

    def DefineTotalTransport(self, currentSolution, withMinimumDistance = False):
        # 这个函数需要再solver中调用，并且能同时输出考虑MinimumDistance和不考虑的结果
        if withMinimumDistance == False:
            totalTransport = (currentSolution.TransportMatrix * currentSolution.ArrangedDMatrix).sum()
        else:
            totalTransport = (currentSolution.TransportMatrix * currentSolution.UpdatedDMatrix).sum()
        currentSolution.TotalTransport = totalTransport

        return totalTransport




el = EvaluationLogic()

el.DefineTotalTransport(sol_1)
print(sol_1)

el.DefineTotalTransport(sol_1, True)
print(sol_1)



el.DefineTotalTransport(sol_2)
print(sol_2)

el.DefineTotalTransport(sol_2, True)
print(sol_2)

el.DefineTotalTransport(sol_3)
print(sol_3)
el.DefineTotalTransport(sol_3, True)
print(sol_3)


Die Permutation [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11] führt zu einer Transportleistung von 50116
Die Permutation [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11] führt zu einer Transportleistung von 50998
Die Permutation [0, 11, 1, 10, 2, 9, 3, 8, 4, 7, 5, 6] führt zu einer Transportleistung von 63118
Die Permutation [0, 11, 1, 10, 2, 9, 3, 8, 4, 7, 5, 6] führt zu einer Transportleistung von 63568
Die Permutation [0, 2, 4, 6, 8, 10, 11, 9, 7, 5, 3, 1] führt zu einer Transportleistung von 45322
Die Permutation [0, 2, 4, 6, 8, 10, 11, 9, 7, 5, 3, 1] führt zu einer Transportleistung von 46102


In [94]:
print([*range(len(output_12.D))])
print(list(range(12)))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]


In [124]:
# 到这里需要用启发式算法生成一个startSolution，这个方案要尽可能接近最优解
# 我选择的方法是ROS，随机数，即设置一个随机数种子，然后生成多个随机排列，每个排列包含的元素个数等于Size
# 首先尝试使用ROS

# class ConstructiveHeuristics:
#     def __init__(self, evaluationLogic, solutionPool):
#         self.RandomSeed = 2022
#         self.RandomRetiris = 10
#         self.EvaluationLogic = evaluationLogic
#         self.SolutionPool = solutionPool
import numpy 
# from EvaluationLogic import *

def ROS(outputData, iterations, seed):
    numpy.random.seed(seed)
    tmpSolution = Solution(outputData, outputData.InitialPermutation)
    bestCmax = numpy.inf

   
    for i in range(iterations):
        tmpPermutation = numpy.random.permutation(len(outputData.D))  # 随机排列能正常生成，
        tmpSolution.setPermutation(tmpPermutation)
        # tmpSolution.Permutation = tmpPermutation

        el = EvaluationLogic()
        el.DefineTotalTransport(tmpSolution, False)  # 这一行运行错误

        if (tmpSolution.TotalTransport < bestCmax):
            bestCmax = tmpSolution.TotalTransport
            bestPerm = tmpPermutation
     

    bestSol = Solution(outputData, bestPerm)
    el.DefineTotalTransport(bestSol, False)

    return bestSol



In [125]:
print(output_12.InitialPermutation)
print(output_12.D)
print(len(output_12.D))

s = Solution(output_12, output_12.InitialPermutation)
EvaluationLogic().DefineTotalTransport(s)
print(s)

print(ROS(output_12, 12, 2022))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
[[0 1 2 3 1 2 3 4 2 3 4 5]
 [1 0 1 2 2 1 2 3 3 2 3 4]
 [2 1 0 1 3 2 1 2 4 3 2 3]
 [3 2 1 0 4 3 2 1 5 4 3 2]
 [1 2 3 4 0 1 2 3 1 2 3 4]
 [2 1 2 3 1 0 1 2 2 1 2 3]
 [3 2 1 2 2 1 0 1 3 2 1 2]
 [4 3 2 1 3 2 1 0 4 3 2 1]
 [2 3 4 5 1 2 3 4 0 1 2 3]
 [3 2 3 4 2 1 2 3 1 0 1 2]
 [4 3 2 3 3 2 1 2 2 1 0 1]
 [5 4 3 2 4 3 2 1 3 2 1 0]]
12
Die Permutation [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11] führt zu einer Transportleistung von 50116
Die Permutation [ 3  5  9  4  6  2  8  7 11 10  1  0] führt zu einer Transportleistung von 63306


In [77]:
import numpy as np
np.random.seed(2022)
x = 0
while x < 10:
    perm = np.random.permutation(5)
    print(perm)
    x+=1

[2 3 0 1 4]
[4 2 3 0 1]
[3 2 1 4 0]
[4 0 2 3 1]
[2 1 4 3 0]
[3 4 1 2 0]
[0 1 3 4 2]
[3 4 1 0 2]
[0 2 4 3 1]
[1 3 2 0 4]
